In [1]:
                                     import pandas  as pd
                                     import numpy   as np
                                     import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise        import cosine_similarity

In [2]:
movies = pd.read_csv('movies.csv')

In [3]:
movies.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movies.shape

(4803, 24)

In [5]:
movies.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [6]:
needed_cols = ['genres', 'keywords', 'tagline', 'cast', 'director']
df = movies[needed_cols]

df.isnull().sum()

genres       28
keywords    412
tagline     844
cast         43
director     30
dtype: int64

In [7]:
for col in needed_cols:
  df[col] = df[col].fillna('')

df.isnull().sum()

/var/folders/n0/_5ytbzzj5tqdxd4yymn1bkhw0000gn/T/ipykernel_20547/1220386487.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna('')


genres      0
keywords    0
tagline     0
cast        0
director    0
dtype: int64

In [8]:
# Combiner les features
combined_cols = df['genres'] + " " + df['keywords'] + " " + df['tagline'] + " " + df['cast'] + " " + df['director']
combined_cols

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [9]:
# Que contient la première ligne ?
combined_cols[0]

'Action Adventure Fantasy Science Fiction culture clash future space war space colony society Enter the World of Pandora. Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez James Cameron'

In [10]:
# convertir en nombres en utilisant la fonction `TfidfVectorizer()`
vectorizer = TfidfVectorizer()
vectorized_combined_cols = vectorizer.fit_transform(combined_cols)

In [11]:
# À quoi ça ressemble ?
np.set_printoptions(threshold=np.inf)
vectorized_combined_cols.toarray()[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [12]:
similarity = cosine_similarity(vectorized_combined_cols)
np.set_printoptions(threshold=10)   # n'imprimera que les 10 premières lignes du tableau numpy
similarity

array([[1.        , 0.07219487, 0.037733  , ..., 0.        , 0.        ,
        0.        ],
       [0.07219487, 1.        , 0.03281499, ..., 0.03575545, 0.        ,
        0.        ],
       [0.037733  , 0.03281499, 1.        , ..., 0.        , 0.05389661,
        0.        ],
       ...,
       [0.        , 0.03575545, 0.        , ..., 1.        , 0.        ,
        0.02651502],
       [0.        , 0.        , 0.05389661, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02651502, 0.        ,
        1.        ]], shape=(4803, 4803))

In [13]:
# Lister les films
movie_titles = movies['title'].tolist()
movie_titles

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [14]:
# Trouver les mots analogues
user_input = input("Entre ton film préféré pour obtenir des recommandations similaires : ")
close_movie_names = difflib.get_close_matches(user_input, movie_titles)
close_movie_names

Entre ton film préféré pour obtenir des recommandations similaires :  iron man 3


['Iron Man 3', 'Iron Man 2', 'Iron Man']

In [15]:
# Les premiers noms sont plus pertinents
close_match = close_movie_names[0]
close_match

'Iron Man 3'

In [16]:
# Quelle est la valeur de l'index du film ?
index_of_the_movie = movies[movies.title == close_match]['index'].values[0]
index_of_the_movie

np.int64(31)

In [17]:
# Quel est le score de similarité ?
similarity_score = list(enumerate(similarity[index_of_the_movie]))
similarity_score

[(0, np.float64(0.060555221380551486)),
 (1, np.float64(0.04517303238040466)),
 (2, np.float64(0.025587599816574432)),
 (3, np.float64(0.0596552556152424)),
 (4, np.float64(0.04491082754059967)),
 (5, np.float64(0.024826539308291366)),
 (6, np.float64(0.008095033555461727)),
 (7, np.float64(0.22661385170966214)),
 (8, np.float64(0.00826062907345574)),
 (9, np.float64(0.05587767668577866)),
 (10, np.float64(0.05070248377744624)),
 (11, np.float64(0.012498348966608385)),
 (12, np.float64(0.01436538661417187)),
 (13, np.float64(0.02209315018430714)),
 (14, np.float64(0.0870997625678635)),
 (15, np.float64(0.01808085308279186)),
 (16, np.float64(0.21487810271500668)),
 (17, np.float64(0.01374215999080098)),
 (18, np.float64(0.03542729148083272)),
 (19, np.float64(0.029365168971596863)),
 (20, np.float64(0.09349691478373993)),
 (21, np.float64(0.011807521103905674)),
 (22, np.float64(0.016161416228396827)),
 (23, np.float64(0.02403080353597918)),
 (24, np.float64(0.055063171749892295)),
 (2

In [18]:
# Trier par décroissancce pour que les similarités soient en haut de la liste
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
sorted_similar_movies

[(31, np.float64(1.0)),
 (79, np.float64(0.49790727997578726)),
 (68, np.float64(0.3146705244947751)),
 (7, np.float64(0.22661385170966214)),
 (26, np.float64(0.22176049914086782)),
 (16, np.float64(0.21487810271500668)),
 (182, np.float64(0.16398518362587794)),
 (46, np.float64(0.16100016953381988)),
 (2625, np.float64(0.15868163752956252)),
 (4401, np.float64(0.15202049419143118)),
 (511, np.float64(0.15049377529939592)),
 (356, np.float64(0.15008507803087165)),
 (33, np.float64(0.1481532095189962)),
 (64, np.float64(0.14590707709113576)),
 (203, np.float64(0.14246252089473255)),
 (466, np.float64(0.13913012151683268)),
 (205, np.float64(0.13575975331420798)),
 (607, np.float64(0.1346030776822172)),
 (1439, np.float64(0.1340667758394713)),
 (174, np.float64(0.13262656259706457)),
 (85, np.float64(0.13136489442315136)),
 (38, np.float64(0.12918737550877274)),
 (2487, np.float64(0.1290042198638991)),
 (788, np.float64(0.12630434059294124)),
 (126, np.float64(0.12590649403009993)),
 (12

In [19]:
# affichier les 30 films analogues à Iron man 2
print("Films pouvants t'intéresser : \n")
i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies[movies.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Films pouvants t'intéresser : 

1 . Iron Man 3
2 . Iron Man 2
3 . Iron Man
4 . Avengers: Age of Ultron
5 . Captain America: Civil War
6 . The Avengers
7 . Ant-Man
8 . X-Men: Days of Future Past
9 . Kiss Kiss Bang Bang
10 . The Helix... Loaded
11 . X-Men
12 . Sherlock Holmes
13 . X-Men: The Last Stand
14 . X-Men: Apocalypse
15 . X2
16 . The Time Machine
17 . Sherlock Holmes: A Game of Shadows
18 . Sky Captain and the World of Tomorrow
19 . Lions for Lambs
20 . The Incredible Hulk
21 . Captain America: The Winter Soldier
22 . The Amazing Spider-Man 2
23 . Duets
24 . Deadpool
25 . Thor: The Dark World
26 . X-Men Origins: Wolverine
27 . X-Men: First Class
28 . Shallow Hal
29 . Captain America: The First Avenger
